In [4]:
import numpy as np
from tqdm import tqdm
import csv

In [5]:
#隐马尔可夫模型
#面向命名实体识别(组织机构)问题
class HMM_BIO_main:
    def __init__(self,A=None,B=None,PI=None):
        #param:
        # (A,B,PI) 状态转移矩阵 发射矩阵 初始状态矩阵
        self.print=True
        
        
        self.N = 3     # 状态集合有多少元素(状态数BIO)
        self.M = 65535 # 观测集合有多少元素(汉字)
        
        #数字化 状态集合Q 观测集合V 
        self.status_dict={ # {'盒子1':0，'盒子2':1，'盒子3':2}
                        'B-ORG': 0,
                        'I-ORG': 1,
                        'O': 2}
        self.status=list(self.status_dict.keys())          
        #self.observe_dict={}# {'红':0，'白':1}
        self.Q=np.arange(0,self.N) #状态集合 [0,1,2]
        self.V=np.arange(0,self.M) #观测集合 [0,1]
        
        #初始化 (A,B,PI)
        self.A=A # 状态转移矩阵
        self.B=B # 发射矩阵
        self.PI=PI # 初始状态概率
        
        
        if self.print:
            print('状态集合status',self.status_dict)
            #print('观测集合observe',self.observe_dict)
            print('状态集合Q',self.Q)
            print('观测集合V',self.V)
            print()
        

    
    #训练模型(A,B,PI)
    #data (2,N)
    def train(self,filename,savedir=None):
        #利用数据data(状态序列+观测序列) 计算参数(A,B,PI) 直接使用统计方法 统计A,B,PI
        
        A=np.zeros((self.N,self.N)) #(N,N)
        B=np.zeros((self.N,self.M)) #(N,M)
        PI=np.zeros(self.N) #(N,)
        
        
        with open(filename,'r',encoding='utf-8') as fr:
            mult=10000000
            cnt=0
            
            line=fr.readline()
            pres=-1
            while line:
        
                mult-=1
                if mult==0:
                    cnt+=1
                    print(f'load {10000000}*{cnt} rows')
                    mult=10000000
                    
                
                
                s,v=line.split(',')
                s=int(s)
                v=int(v[:-1])
                #print(line,s,v)
                #当前为分隔符
                if s==-1 or v>=self.M:
                    pres=-1
                    line=fr.readline()
                    continue
                if pres!=-1:
                    #s 当前状态
                    #pres 上一次状态
                    #v 当前观测

                    A[pres,s]+=1
                    B[s,v]+=1
                    
                #上一个为分隔符
                else:
                    PI[s]+=1
                    B[s,v]+=1
                pres=s
                line=fr.readline()
        
        self.A=A
        self.B=B
        self.PI=PI
        print('参数统计结果(A,PI)')
        print(f'A:{self.A}')
        print(f'PI:{self.PI}')
        
        
        self.normalize_param()
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.PI}')
        
        
        self.save_Param(savedir)
        
        
    
    def normalize_param(self):
        epsilon=1e-8
        
        
        self.PI[self.PI == 0] =epsilon  # 防止数据下溢,对数据进行对数归一化
        self.PI = np.log(self.PI) - np.log(np.sum(self.PI))

        self.A[self.A == 0] = epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))

        self.B[self.B == 0] = epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        
        #将(A,B,PI)归一化 (直接归一化)
        #self.A=self.A/np.sum(self.A,axis=1,keepdims=True)
        #self.B=self.B/np.sum(self.B,axis=1,keepdims=True)
        #self.PI=self.PI/np.sum(self.PI)

    def save_Param(self,savedir):
        if savedir==None:
            savedir='param/BIO/main/'
        np.savetxt(f'{savedir}A.csv',self.A,delimiter=',')
        np.savetxt(f'{savedir}B.csv',self.B,delimiter=',')
        np.savetxt(f'{savedir}PI.csv',self.PI,delimiter=',')
        print(f'训练参数已保存在 {savedir}A.csv')
        print()
    
    def load_Param(self,loaddir='param/BIO/main/'):
        self.A=np.genfromtxt(f'{loaddir}A.csv',delimiter=',')
        self.B=np.genfromtxt(f'{loaddir}B.csv',delimiter=',')
        self.PI=np.genfromtxt(f'{loaddir}PI.csv',delimiter=',')
        print(f'训练参数(A,B,PI)已从 {loaddir}A.csv 中读取成功')
        print()
    
    def viterbi_t(self,o):
        #维特比算法(动态规划)
        #已知(A,B,PI) 和一观测序列o 求解其状态序列的概率最大解
        #param:
        # o 观测序列 [0,1,1,1,1,0] shape(n,)
        #return:
        # ret 状态序列 shape(n,)
        #solution:
        # delta计算从前一个的所有状态 到当前状态的概率最大值 psi记录下来概率最大值的前一个状态
        # 从delta的最后取最大值，利用psi向前回溯即可找到最大概率序列
        
        n=o.shape[0]
        
        delta=np.zeros((self.N,n)) #delta[:,i] 为到达该观测序列的 每个状态的最大概率值 shape(状态个数,序列长度)
        psi=np.zeros((self.N,n),dtype=np.int32) #psi[:,i] 为到达该观测序列的 最大概率值的 上一个状态为哪个
        A=self.A
        B=self.B
        PI=self.PI
        
        #* -> + (因为log)
        delta[:,0]=PI+B[:,o[0]]
        #psi[:,0]=np.argmax(delta[:,0])
        for i in range(1,n):
            temp=delta[:,i-1]+A.T
            psi[:,i]=np.argmax(temp,axis=1)
            delta[:,i]=np.max(temp,axis=1)+B[:,o[i]]
        
        ret=np.zeros(n,dtype=np.int32)
        ret[-1]=np.argmax(delta[:,-1])

        for i in range(n-2,-1,-1):
            ret[i]=psi[ret[i+1]][i+1]
        return ret
    
    
    
    def calc_acc(self,label,predict):
        #计算标签预测的准确率
        #param:
        # a 源标签
        # b 目的标签
        return np.mean(np.equal(label,predict))
    
    
    
    def predict(self,s:str):
        #预测一段话的状态序列
        #param
        # s 一段中文
        #print
        # 中文加状态的文字 林B-LOC_|徽I-LOC_|因_O_|
        
        ans=""
        s=list(s)
        o=np.array([ord(ch) for ch in s])
        #打印观测序列
        #print(o)
        statusList=self.viterbi_t(o)
        for i in range(len(s)):
            ans+=s[i]+str(self.status[statusList[i]])+'_|'
        #print(statusList)
        print(ans)
        

In [6]:
#将默认的语料库 中的观测序列和状态序列转换为数字存入csv文件中 (可以用np格式读出)
def data_pretreatment(hmm:HMM_BIO_main):
    file_path='corpus_part0.txt'
    with open(file_path,mode='r',encoding='utf-8') as fr:
        with open('BIO_train_pretreatment_0.csv',mode='w',encoding='utf-8',newline='') as fw:
            writer=csv.writer(fw)
            line=fr.readline()
            cnt=10000000
            while line:
                #判断是否是换行\n
                if len(line)==1:
                    writer.writerow([-1,-1])
                else:
                    #print(line)
                    ch,tag=line[0],line[2:-1]
                    writer.writerow([hmm.status_dict[tag],ord(ch)])
                    #new_row=np.array([[hmm.status_dict[tag],ord(ch)]]) 
                line=fr.readline()
                
                cnt-=1
                if cnt==0:
                    print('write 1000,0000 rows')
                    cnt=10000000
            
                
#查看数组a1的属性
def check(a1):
    print(a1)  
    print("数据类型",type(a1))           #打印数组数据类型  
    print("数组元素数据类型：",a1.dtype) #打印数组元素数据类型  
    print("数组元素总数：",a1.size)      #打印数组尺寸，即数组元素总数  
    print("数组形状：",a1.shape)         #打印数组形状  
    print("数组的维度数目",a1.ndim)      #打印数组的维度数目
    print()

In [7]:
#创建一个HMM模型用于命名实体识别
hmm_bio=HMM_BIO_main()

状态集合status {'B-ORG': 0, 'I-ORG': 1, 'O': 2}
状态集合Q [0 1 2]
观测集合V [    0     1     2 ... 65532 65533 65534]



In [ ]:
#将txt文本数据预处理 写进csv文件
data_pretreatment(hmm_bio)

In [8]:
#根据已有数据 训练HMM模型
hmm_bio.train('corpus/BIO_train_pretreatment_1.csv')

参数统计结果(A,PI)
A:[[  0.  17.   0.]
 [  0. 122.  10.]
 [ 10.   0. 664.]]
PI:[ 7.  0. 88.]
训练参数结果(A,B,PI)
A:[[-2.12538941e+01 -1.17647048e-09 -2.12538941e+01]
 [-2.33034827e+01 -7.87808779e-02 -2.58021683e+00]
 [-4.21064502e+00 -2.49339109e+01 -1.49479615e-02]]
B:[[-21.25393263 -21.25393263 -21.25393263 ... -21.25393263 -21.25393263
  -21.25393263]
 [-23.35515939 -23.35515939 -23.35515939 ... -23.35515939 -23.35515939
  -23.35515939]
 [-25.05662816 -25.05662816 -25.05662816 ... -25.05662816 -25.05662816
  -25.05662816]]
PI:[ -2.60796674 -22.97455764  -0.07654008]
训练参数已保存在 param/BIO/main/A.csv



In [27]:
#进行预测
sentences=['龙川县博物馆馆藏文物','林徽因什么理由拒绝了徐志摩而选择梁思成变为终身伴侣?谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分','王心雷编写了一个代码',
      '王康在哈尔滨出门打电动被张舒帆当场逮捕','北京抖音信息服务有限公司今天出台了一个新的政策','中国招标网发布了一个新的招标',
       '我真的好饿','中标人:遂宁高新区蜜感内衣店','中国是个好地方','宁夏最高气温20度','恭喜安居区安居镇有家超市昨天开业了',
           '家乐福是个好地方','家乐福','家乐福超市']
for sentence in sentences:
    hmm_bio.predict(sentence)
    print()


龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|

林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐I-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思I-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，I-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

王B-ORG_|心I-ORG_|雷I-ORG_|编O_|写O_|了O_|一O_|个O_|代O_|码O_|

王O_|康B-ORG_|在I-ORG_|哈I-ORG_|尔I-ORG_|滨I-ORG_|出O_|门B-ORG_|打I-ORG_|电I-ORG_|动I-ORG_|被I-ORG_|张I-ORG_|舒I-ORG_|帆I-ORG_|当I-ORG_|场I-ORG_|逮I-ORG_|捕I-ORG_|

北B-ORG_|京I-ORG_|抖I-ORG_|音I-ORG_|信O_|息O_|服O_|务B-ORG_|有I-ORG_|限I-ORG_|公I-ORG_|司I-ORG_|今I-ORG_|天I-ORG_|出O_|台O_|了O_|一O_|个O_|新O_|的O_|政B-ORG_|策I-ORG_|

中O_|国B-ORG_|招I-ORG_|标I-ORG_|网I-ORG_|发I-ORG_|布O_|了O_|一O_|个O_|新B-ORG_|的I-ORG_|招I-ORG_|标I-ORG_|

我B-ORG_|真I-ORG_|的O_|好B-ORG_|饿I-ORG_|

中O_|标O_|人O_|:O_|遂B-ORG_|宁I-ORG_|高I-ORG_|新I-ORG_|区O_|蜜O_|感O_|内O_|衣B-ORG_|店I-ORG_|

中O_|国B-ORG_|是I-ORG_|个O_|

In [7]:
#哈尔滨 中招国联 科技 有限公司 北京分公司
#Ns地域
#NN 名号
#NM 领域
#NO 组织形态
#NB 分支机构

In [109]:
print(chr(128072))

👈
